In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

In [2]:
spark = SparkSession.builder.appName("pleio_and_filter").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/16 15:44:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/16 15:44:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
chromosome="NONE"
chromosome="chr22"

if "which_chr" in os.environ:
    chromosome=os.environ['which_chr']

if chromosome=="NONE":
    print("error : did not find which chromosome we are supposed to crunch")
    exit(-1)
else:
    print("only crunching chromosome "+chromosome)

only crunching chromosome chr22


In [4]:
variant_path=f"/home/mcn26/varef/scripts/noon_data/2.5.filter/{chromosome}.csv.gz/*.csv.gz"
variants=spark.read.option("delimiter","\t").csv(variant_path, header=True, inferSchema=True)

24/05/16 15:44:38 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Dropping columns with bad allele frequencies

In [14]:
variants=variants.filter(F.col("category")!="MAF_OR_AC_IS_ZERO")

## Pleitropy

"Pleitropy" here refers to a variant which is an emVar in multiple cell-types. We're calling emVars as anything with abs(skew)>=0.5 and max(alt activity, ref activity)>=1

In [15]:
#first we compute whether each variant can be called an emvar in each cell-type. 
cell_types=["K562","SKNSH","HepG2"]
for cell_type in cell_types:
    variants = variants.withColumn(f"emVar_{cell_type}", 
                           (F.abs(F.col(f"{cell_type}__skew")) >= 0.5) & 
                           (F.greatest(F.col(f"{cell_type}__ref"), F.col(f"{cell_type}__alt")) >= 1.0))

#next, we count the number of cell-types each variant is an emvar in to compute the pleitropy. 
variants = variants.withColumn("pleio", F.col("emVar_K562").cast("int") + F.col("emVar_SKNSH").cast("int") + F.col("emVar_HepG2").cast("int"))

In [ ]:
output_root="/home/mcn26/varef/scripts/noon_data/3.0pleio_and_filter/"

variants.write \
    .option("header","true") \
    .option("delimiter","\t") \
    .option("compression", "gzip") \
    .csv(output_root+chromosome)

spark.stop()